# Imports and installations

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 5.2 MB/s 
     |████████████████████████████████| 120 kB 57.3 MB/s 
     |████████████████████████████████| 6.6 MB 38.8 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 365 kB 5.1 MB/s 
     |████████████████████████████████| 115 kB 62.1 MB/s 
     |████████████████████████████████| 212 kB 49.9 MB/s 
     |████████████████████████████████| 127 kB 63.4 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import concatenate_datasets, load_dataset, Dataset
from transformers import (AutoModelWithLMHead,
                          AutoTokenizer,
                          TFAutoModelWithLMHead,
                          TFAutoModelForCausalLM,
                          AutoModelForCausalLM,
                          AutoConfig,
                          Trainer,
                          TextDataset,
                          DataCollatorForLanguageModeling,
                          TrainingArguments,
                          TrainerCallback,
                          pipeline
)

import os
from tqdm import tqdm

# CREATING AND PROCESSING DATASET PAWS-x

## Loading datasets

In [ ]:
dataset_en = load_dataset("paws-x",'en')

Generating train split:   0%|          | 0/49401 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Dataset pawsx downloaded and prepared to /root/.cache/huggingface/datasets/pawsx/en/1.1.0/37673404a6de6d0fa2574661e77940d10d3be3bf51bb4f08c8fa079fd56b9755. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
dataset_es = load_dataset("paws-x",'es')

Generating train split:   0%|          | 0/49401 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Dataset pawsx downloaded and prepared to /root/.cache/huggingface/datasets/pawsx/es/1.1.0/37673404a6de6d0fa2574661e77940d10d3be3bf51bb4f08c8fa079fd56b9755. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
dataset_es

DatasetDict({
    train: Dataset({
        features: ['id', 'sentence1', 'sentence2', 'label'],
        num_rows: 49401
    })
    test: Dataset({
        features: ['id', 'sentence1', 'sentence2', 'label'],
        num_rows: 2000
    })
    validation: Dataset({
        features: ['id', 'sentence1', 'sentence2', 'label'],
        num_rows: 2000
    })
})

In [ ]:
dataset_en

DatasetDict({
    train: Dataset({
        features: ['id', 'sentence1', 'sentence2', 'label'],
        num_rows: 49401
    })
    test: Dataset({
        features: ['id', 'sentence1', 'sentence2', 'label'],
        num_rows: 2000
    })
    validation: Dataset({
        features: ['id', 'sentence1', 'sentence2', 'label'],
        num_rows: 2000
    })
})

## Concatenate default datasets

In [ ]:
dataset_es = concatenate_datasets([dataset_es['train'], dataset_es['train'], dataset_es['validation']])
dataset_en = concatenate_datasets([dataset_en['train'], dataset_en['train'], dataset_en['validation']])

In [ ]:
dataset_en

Dataset({
    features: ['id', 'sentence1', 'sentence2', 'label'],
    num_rows: 100802
})

In [ ]:
lan_es_column = ['es'] * len(dataset_es)
lan_en_column = ['en'] * len(dataset_en)
dataset_es = dataset_es.add_column("lan", lan_es_column )
dataset_en = dataset_en.add_column("lan", lan_en_column)

In [ ]:
dataset_comb = concatenate_datasets([dataset_es, dataset_en])

In [ ]:
dataset_comb

Dataset({
    features: ['id', 'sentence1', 'sentence2', 'label', 'lan'],
    num_rows: 201604
})

## Filtering

In [ ]:
dataset_comb = dataset_comb.filter(lambda example: example['label'] == 1)
print(dataset_comb)

  0%|          | 0/202 [00:00<?, ?ba/s]

Dataset({
    features: ['id', 'sentence1', 'sentence2', 'label', 'lan'],
    num_rows: 89026
})


## Shuffling

In [ ]:
PAWS_X =  dataset_comb.shuffle(seed=42)

## To dataframe

In [ ]:
PAWS_X = pd.DataFrame(PAWS_X, index = PAWS_X['id'] )


In [ ]:
PAWS_X = PAWS_X.drop('id', 1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """Entry point for launching an IPython kernel.


In [ ]:
PAWS_X

,sentence1,sentence2,label,lan
19990,La esposa de Quinatzin era una princesa de Hue...,La esposa de Quinatzin era una princesa de Hue...,1,es
10801,"The academy consists of east hall , central wi...","The academy consists of eastern hall , central...",1,en
20285,Critical Millennium is a 2010 graphic novel pu...,Critical Millennium is a novel published by Ar...,1,en
39049,Stenolechia zelosaris is a moth of the family ...,Stenolechia zelosaris is a moth from the famil...,1,en
28878,Gary Lucy was paired with Vanilla Ice in serie...,Gary Lucy was paired with Vanilla Ice in serie...,1,en
...,...,...,...,...
29788,"El 30 de agosto de 1853, la hija de Stephen Ch...","El 30 de agosto de 1853, la hija de Stephen Ch...",1,es
8953,Other types of course offered by the Departmen...,Other types of courses offered by the departme...,1,en
12708,"Algunos tienen efectos negativos, algunos posi...",Algunos tienen efectos negativos y algunos pos...,1,es
605,"Con el debilitamiento del dólar canadiense, la...","En noviembre y diciembre de 2015, con el debil...",1,es


In [ ]:
PAWS_X = PAWS_X.drop('label', 1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """Entry point for launching an IPython kernel.


In [ ]:
PAWS_X = PAWS_X.reset_index(drop=True)


In [ ]:
PAWS_X

,sentence1,sentence2,lan
0,La esposa de Quinatzin era una princesa de Hue...,La esposa de Quinatzin era una princesa de Hue...,es
1,"The academy consists of east hall , central wi...","The academy consists of eastern hall , central...",en
2,Critical Millennium is a 2010 graphic novel pu...,Critical Millennium is a novel published by Ar...,en
3,Stenolechia zelosaris is a moth of the family ...,Stenolechia zelosaris is a moth from the famil...,en
4,Gary Lucy was paired with Vanilla Ice in serie...,Gary Lucy was paired with Vanilla Ice in serie...,en
...,...,...,...
89021,"El 30 de agosto de 1853, la hija de Stephen Ch...","El 30 de agosto de 1853, la hija de Stephen Ch...",es
89022,Other types of course offered by the Departmen...,Other types of courses offered by the departme...,en
89023,"Algunos tienen efectos negativos, algunos posi...",Algunos tienen efectos negativos y algunos pos...,es
89024,"Con el debilitamiento del dólar canadiense, la...","En noviembre y diciembre de 2015, con el debil...",es


## Saving dataset

In [ ]:
PAWS_X.to_csv(r'/content/drive/MyDrive/working directory/tfm/Distilled GPT2 finetuning/datasets/PAWSX/PAWS_X.csv',index=False)


# CREATING AND PROCESSING DATASET TAPACO

## Loading datasets

In [ ]:
tapaco_es = load_dataset("tapaco", "es")
tapaco_en = load_dataset("tapaco", "en")

Reusing dataset tapaco (/root/.cache/huggingface/datasets/tapaco/es/1.0.0/71d200534b520a174927a8f0479c06220a0a6fb5201a84ebfce19006c6354698)


  0%|          | 0/1 [00:00<?, ?it/s]

Reusing dataset tapaco (/root/.cache/huggingface/datasets/tapaco/en/1.0.0/71d200534b520a174927a8f0479c06220a0a6fb5201a84ebfce19006c6354698)


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
tapaco_es['train']

Dataset({
    features: ['paraphrase_set_id', 'sentence_id', 'paraphrase', 'lists', 'tags', 'language'],
    num_rows: 85064
})

In [ ]:
tapaco_en['train']

Dataset({
    features: ['paraphrase_set_id', 'sentence_id', 'paraphrase', 'lists', 'tags', 'language'],
    num_rows: 158053
})

## Processsing datasets

In [ ]:


#Loading dataset
dataset1 = load_dataset('tapaco', 'en')
dataset2 = load_dataset('tapaco', 'es')
def process_tapaco_dataset(dataset, out_file):
    tapaco = []
    # The dataset has only train split.
    for data in tqdm(dataset["train"]):
        keys = data.keys()
        tapaco.append([data[key] for key in keys])
    tapaco_df = pd.DataFrame(
        data=tapaco,
        columns=[
            "paraphrase_set_id",
            "sentence_id 	",
            "paraphrase",
            "lists", 
            "tags", 
            "language",
        ],
    )
    tapaco_df.to_csv(out_file, sep="\t", index=None)
    return tapaco_df

tapaco_df_en = process_tapaco_dataset(dataset1,r"/content/drive/MyDrive/working directory/tfm/Distilled GPT2 finetuning/datasets/tapaco/tapaco_en.csv")
tapaco_df_es = process_tapaco_dataset(dataset2,r"/content/drive/MyDrive/working directory/tfm/Distilled GPT2 finetuning/datasets/tapaco/tapaco_es.csv")


100%|██████████| 85064/85064 [00:18<00:00, 4523.44it/s]


In [ ]:
def generate_tapaco_paraphrase_dataset(dataset):
    dataset_df = dataset[["paraphrase", "paraphrase_set_id"]]
    non_single_labels = (
        dataset_df["paraphrase_set_id"]
        .value_counts()[dataset_df["paraphrase_set_id"].value_counts() > 1]
        .index.tolist()
    )
    tapaco_df_sorted = dataset_df.loc[
        dataset_df["paraphrase_set_id"].isin(non_single_labels)
    ]
    tapaco_paraphrases_dataset = []

    for paraphrase_set_id in tqdm(tapaco_df_sorted["paraphrase_set_id"].unique()):
        id_wise_paraphrases = tapaco_df_sorted[
            tapaco_df_sorted["paraphrase_set_id"] == paraphrase_set_id
        ]
        len_id_wise_paraphrases = (
            id_wise_paraphrases.shape[0]
            if id_wise_paraphrases.shape[0] % 2 == 0
            else id_wise_paraphrases.shape[0] - 1
        )
        for ix in range(0, len_id_wise_paraphrases, 2):
            current_phrase = id_wise_paraphrases.iloc[ix][0]
            for count_ix in range(ix + 1, ix + 2):
                next_phrase = id_wise_paraphrases.iloc[ix + 1][0]
                tapaco_paraphrases_dataset.append([current_phrase, next_phrase])
    tapaco_paraphrases_dataset_df = pd.DataFrame(
        tapaco_paraphrases_dataset, columns=["Text", "Paraphrase"]
    )
 
    return tapaco_paraphrases_dataset_df
    
tapaco_paraphrases_dataset_df_es = generate_tapaco_paraphrase_dataset(tapaco_df_es)
tapaco_paraphrases_dataset_df_en = generate_tapaco_paraphrase_dataset(tapaco_df_en)

100%|██████████| 62044/62044 [17:22<00:00, 59.50it/s]


In [ ]:
tapaco_paraphrases_dataset_df_es['lan'] = ['es'] * len(tapaco_paraphrases_dataset_df_es)
tapaco_paraphrases_dataset_df_en['lan'] = ['en'] * len(tapaco_paraphrases_dataset_df_en)

## Concatenat en es datasets

In [ ]:
tapaco_x = pd.concat([tapaco_paraphrases_dataset_df_en, tapaco_paraphrases_dataset_df_es], ignore_index=True, sort=False)

## Shuffling

In [ ]:
tapaco_x = tapaco_x.sample(frac=1)

In [ ]:
tapaco_x = tapaco_x.reset_index(drop=True)

In [ ]:
tapaco_x.to_csv("/content/drive/MyDrive/working directory/tfm/Distilled GPT2 finetuning/datasets/tapaco/tapaco_x.csv", index=None)

# CREATING AND PROCESSING COMBINED DATASET(PAWSX + Tapaco)

## Load datasets

In [ ]:
  tapaco_x = pd.read_csv("/content/drive/MyDrive/working directory/tfm/Distilled GPT2 finetuning/datasets/tapaco/tapaco_x.csv")
  PAWS_X = pd.read_csv('/content/drive/MyDrive/working directory/tfm/Distilled GPT2 finetuning/datasets/PAWSX/PAWS_X.csv')

In [ ]:
PAWS_X = PAWS_X.rename(columns={'sentence1': 'Text', 'sentence2': 'Paraphrase'})

## Concatenate

In [ ]:
dataset_comb = pd.concat([tapaco_x, PAWS_X], ignore_index=True, sort=False)

In [ ]:
dataset_comb

,Text,Paraphrase,lan
0,¡Dese prisa que perdemos el tren!,¡Apúrate o se nos hace tarde para el tren!,es
1,I squeezed Tom's hand.,I shook Tom's hand.,en
2,Tom didn't know who to believe.,Tom didn't know who he should believe.,en
3,Quemaron un poco de carbón.,Ellos quemaron algún carbón.,es
4,I don't think about you.,I am not thinking of you.,en
...,...,...,...
201527,"El 30 de agosto de 1853, la hija de Stephen Ch...","El 30 de agosto de 1853, la hija de Stephen Ch...",es
201528,Other types of course offered by the Departmen...,Other types of courses offered by the departme...,en
201529,"Algunos tienen efectos negativos, algunos posi...",Algunos tienen efectos negativos y algunos pos...,es
201530,"Con el debilitamiento del dólar canadiense, la...","En noviembre y diciembre de 2015, con el debil...",es


## Shuffling

In [ ]:
dataset_comb = dataset_comb.sample(frac=1)
dataset_comb = dataset_comb.reset_index(drop=True)

In [ ]:
dataset_comb

,Text,Paraphrase,lan
0,El gato se come al pequeño ratón.,El gato se comió al ratón.,es
1,"Fue escrito por Nancy Steen y Neil Thompson, d...","Dirigida por Paul Krasny, escrita por Nancy St...",es
2,Where is the coffee shop?,Where is the cafeteria?,en
3,Everybody was startled.,Everybody was stunned.,en
4,Navegó por Sydney y llegó a Río de Janeiro el ...,Navegó por Sydney y llegó el 14 de septiembre ...,es
...,...,...,...
201527,¿Recuerdas quién soy yo?,¿Me reconoces aún?,es
201528,Are you making him a sandwich?,Are you making her a sandwich?,en
201529,Ross - Bridge is a historic bridge in the city...,Ross Bridge is an historic bridge in the town ...,en
201530,"Sin embargo, Michael Jackson, Prince y Madonna...","Sin embargo, Michael Jackson, Prince y Madonna...",es


## Checking and cleaning NAN

In [ ]:
dataset_comb = dataset_comb.dropna()

In [ ]:
dataset_comb[dataset_comb['Paraphrase'].map(type) != str]

,Text,Paraphrase,lan


In [ ]:
dataset_comb.to_csv('/content/drive/MyDrive/working directory/tfm/Distilled GPT2 finetuning/datasets/Combined(PAWSX + Tapaco)/dataset_comb.csv', index= False)

# FINE TUNING DISTILLED GPT2

## GENERATING DISTILLGTP2 INPUTS

In [ ]:
dataset_comb =  pd.read_csv('/content/drive/MyDrive/working_directory/tfm/Distilled_GPT2_finetuning/datasets/Combined(PAWSX + Tapaco)/dataset_comb.csv')

In [ ]:
dataset_comb = dataset_comb[dataset_comb['lan'] == 'en']

In [ ]:
dataset_comb

,Text,Paraphrase,lan
2,Where is the coffee shop?,Where is the cafeteria?,en
3,Everybody was startled.,Everybody was stunned.,en
5,Did Tom have to do that?,Was Tom forced to do that?,en
6,Tom wouldn't talk to me.,Tom doesn't speak to me.,en
7,Time is running out.,Time flew.,en
...,...,...,...
201519,She cottoned to him soon.,She began to like him right away.,en
201521,He tends to be late for meetings.,He's always late to meetings.,en
201525,Only those who believe in the future believe i...,Only in trusting the future is it possible to ...,en
201528,Are you making him a sandwich?,Are you making her a sandwich?,en


In [ ]:
dataset_comb['combined'] = '<s> ' + dataset_comb.Text + ' </s>'+' === '+'<p> '+ dataset_comb.Paraphrase +' </p>'

In [ ]:
dataset_comb

,Text,Paraphrase,lan,combined
2,Where is the coffee shop?,Where is the cafeteria?,en,<s> Where is the coffee shop? </s> === <p> Whe...
3,Everybody was startled.,Everybody was stunned.,en,<s> Everybody was startled. </s> === <p> Every...
5,Did Tom have to do that?,Was Tom forced to do that?,en,<s> Did Tom have to do that? </s> === <p> Was ...
6,Tom wouldn't talk to me.,Tom doesn't speak to me.,en,<s> Tom wouldn't talk to me. </s> === <p> Tom ...
7,Time is running out.,Time flew.,en,<s> Time is running out. </s> === <p> Time fle...
...,...,...,...,...
201519,She cottoned to him soon.,She began to like him right away.,en,<s> She cottoned to him soon. </s> === <p> She...
201521,He tends to be late for meetings.,He's always late to meetings.,en,<s> He tends to be late for meetings. </s> ===...
201525,Only those who believe in the future believe i...,Only in trusting the future is it possible to ...,en,<s> Only those who believe in the future belie...
201528,Are you making him a sandwich?,Are you making her a sandwich?,en,<s> Are you making him a sandwich? </s> === <p...


In [ ]:
dataset_comb_tr, dataset_comb_test = train_test_split(dataset_comb, test_size=0.2)
dataset_comb_test, dataset_comb_val = train_test_split(dataset_comb_test, test_size=0.5)

In [ ]:
dataset_comb_tr = dataset_comb_tr.reset_index(drop=True)
dataset_comb_test = dataset_comb_test.reset_index(drop=True)
dataset_comb_val = dataset_comb_test.reset_index(drop=True)

In [ ]:
print(len(dataset_comb_tr), len(dataset_comb_test), len(dataset_comb_val))

94431 11804 11804


In [ ]:
dataset_comb_tr[['combined','lan']]

In [ ]:
dataset_comb.to_csv('/content/drive/MyDrive/working_directory/tfm/Distilled_GPT2_finetuning/datasets/Combined(PAWSX + Tapaco)/distilgpt2_input/dataset_comb_input.csv', index = False)
dataset_comb_tr.combined.to_csv('/content/drive/MyDrive/working_directory/tfm/Distilled_GPT2_finetuning/datasets/Combined(PAWSX + Tapaco)/distilgpt2_input/dataset_comb_tr.csv', index = False)
dataset_comb_test.combined.to_csv('/content/drive/MyDrive/working_directory/tfm/Distilled_GPT2_finetuning/datasets/Combined(PAWSX + Tapaco)/distilgpt2_input/dataset_comb_test.csv', index = False)
dataset_comb_val.combined.to_csv('/content/drive/MyDrive/working_directory/tfm/Distilled_GPT2_finetuning/datasets/Combined(PAWSX + Tapaco)/distilgpt2_input/dataset_comb_val.csv', index = False)

In [ ]:
dataset_comb_tr = dataset_comb[['combined']]
dataset_comb_test = dataset_comb[['combined']]
dataset_comb_val = dataset_comb[['combined']]

In [ ]:
dataset_comb_tr

,combined
2,<s> Where is the coffee shop? </s> === <p> Whe...
3,<s> Everybody was startled. </s> === <p> Every...
5,<s> Did Tom have to do that? </s> === <p> Was ...
6,<s> Tom wouldn't talk to me. </s> === <p> Tom ...
7,<s> Time is running out. </s> === <p> Time fle...
...,...
201519,<s> She cottoned to him soon. </s> === <p> She...
201521,<s> He tends to be late for meetings. </s> ===...
201525,<s> Only those who believe in the future belie...
201528,<s> Are you making him a sandwich? </s> === <p...


In [ ]:
dataset_comb.to_csv('/content/drive/MyDrive/working_directory/tfm/Distilled_GPT2_finetuning/datasets/Combined(PAWSX + Tapaco)/distilgpt2_input/dataset_comb_input.txt', sep='\t', index = False)
dataset_comb_tr.combined.to_csv('/content/drive/MyDrive/working_directory/tfm/Distilled_GPT2_finetuning/datasets/Combined(PAWSX + Tapaco)/distilgpt2_input/dataset_comb_tr.txt', sep='\t', index = False)
dataset_comb_test.combined.to_csv('/content/drive/MyDrive/working_directory/tfm/Distilled_GPT2_finetuning/datasets/Combined(PAWSX + Tapaco)/distilgpt2_input/dataset_comb_test.txt', sep='\t', index = False)
dataset_comb_val.combined.to_csv('/content/drive/MyDrive/working_directory/tfm/Distilled_GPT2_finetuning/datasets/Combined(PAWSX + Tapaco)/distilgpt2_input/dataset_comb_val.txt', sep='\t', index = False)

## TRAINING DISTILLED GPT2

## With hugging facetrainer API

In [ ]:
!pip install wandb


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.8 MB 1.3 MB/s 
     |████████████████████████████████| 157 kB 11.5 MB/s 
     |████████████████████████████████| 181 kB 13.9 MB/s 
     |████████████████████████████████| 63 kB 760 kB/s 
     |████████████████████████████████| 157 kB 28.4 MB/s 
     |████████████████████████████████| 157 kB 17.4 MB/s 
     |████████████████████████████████| 157 kB 27.9 MB/s 
     |████████████████████████████████| 157 kB 33.9 MB/s 
     |████████████████████████████████| 156 kB 31.9 MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=0d7dffca22c6d148535d04c3d75d9dd9f4f31045947ea45117e647851275ba86
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built pathtools


In [ ]:
import wandb
from wandb.keras import WandbCallback
wandb.login()


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 

··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
%env WANDB_PROJECT=distillgpt2TrainerAPI

env: WANDB_PROJECT=distillgpt2TrainerAPI


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("distilgpt2")

model = AutoModelForCausalLM.from_pretrained("distilgpt2")

In [ ]:
text_path = '/content/drive/MyDrive/working_directory/tfm/Distilled_GPT2_finetuning/datasets/Combined(PAWSX + Tapaco)/distilgpt2_input/'

In [ ]:
data_collator = DataCollatorForLanguageModeling (tokenizer=tokenizer, mlm=False)

In [ ]:
train_dataset = TextDataset(
tokenizer=tokenizer,
file_path=text_path+'dataset_comb_tr.txt',
block_size=512)

Token indices sequence length is longer than the specified maximum sequence length for this model (5002966 > 1024). Running this sequence through the model will result in indexing errors


In [ ]:
test_dataset = TextDataset(
tokenizer=tokenizer,
file_path=text_path+'dataset_comb_test.txt',
block_size=512)

In [ ]:
eval_dataset = TextDataset(
tokenizer=tokenizer,
file_path=text_path+'dataset_comb_val.txt',
block_size=512)

In [ ]:
#Run 16 optimizing parameters
epochs = 5
batch_size = 8

training_args =TrainingArguments (
      output_dir='/content/drive/MyDrive/working_directory/tfm/Distilled GPT2 finetuning/output_model',
      num_train_epochs=epochs,
      per_device_train_batch_size=batch_size,
      warmup_steps=500,
      save_steps=2000,
      logging_steps=10,
      learning_rate=1e-5,
      
)

In [ ]:
from datasets import load_metric
metric = load_metric('accuracy')

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:

trainer = Trainer(  #AdamW / weigth decay
model=model,
args=training_args,
data_collator=data_collator,
train_dataset=train_dataset,
eval_dataset=eval_dataset
)

In [ ]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 9771
  Num Epochs = 5
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 6110
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


wandb: Currently logged in as: lxrandom. Use `wandb login --relogin` to force relogin


Step,Training Loss
10,2.557400
20,2.457100
30,2.452100
40,2.423000
50,2.368500
60,2.290000
70,2.275000
80,2.176100
90,2.205200
100,2.185300


Saving model checkpoint to /content/drive/MyDrive/working directory/tfm/Distilled GPT2 finetuning/output_model/checkpoint-2000
Configuration saved in /content/drive/MyDrive/working directory/tfm/Distilled GPT2 finetuning/output_model/checkpoint-2000/config.json
Model weights saved in /content/drive/MyDrive/working directory/tfm/Distilled GPT2 finetuning/output_model/checkpoint-2000/pytorch_model.bin
Saving model checkpoint to /content/drive/MyDrive/working directory/tfm/Distilled GPT2 finetuning/output_model/checkpoint-4000
Configuration saved in /content/drive/MyDrive/working directory/tfm/Distilled GPT2 finetuning/output_model/checkpoint-4000/config.json
Model weights saved in /content/drive/MyDrive/working directory/tfm/Distilled GPT2 finetuning/output_model/checkpoint-4000/pytorch_model.bin
Saving model checkpoint to /content/drive/MyDrive/working directory/tfm/Distilled GPT2 finetuning/output_model/checkpoint-6000
Configuration saved in /content/drive/MyDrive/working directory/tfm

TrainOutput(global_step=6110, training_loss=1.5307064488749653, metrics={'train_runtime': 2477.6395, 'train_samples_per_second': 19.718, 'train_steps_per_second': 2.466, 'total_flos': 6382826375086080.0, 'train_loss': 1.5307064488749653, 'epoch': 5.0})

In [ ]:
trainer.save_model()

Saving model checkpoint to /content/drive/MyDrive/working directory/tfm/Distilled GPT2 finetuning/output_model
Configuration saved in /content/drive/MyDrive/working directory/tfm/Distilled GPT2 finetuning/output_model/config.json
Model weights saved in /content/drive/MyDrive/working directory/tfm/Distilled GPT2 finetuning/output_model/pytorch_model.bin


In [ ]:
model = AutoModelWithLMHead.from_pretrained('/content/drive/MyDrive/working_directory/tfm/Distilled_GPT2_finetuning/DistilGPT2_output_model')

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("distilgpt2")

Downloading:   0%|          | 0.00/762 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/0.99M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

In [ ]:
generator = pipeline('text-generation', model=model, tokenizer=tokenizer)

In [ ]:
def clean_paraphrase(input_sentence):
  p = generator('<s> '+input_sentence+' </s> === <p> ')
  return p[0]['generated_text'].split(' </s> === <p> ')[1].split(' </p>')[0]

## Testing training With tensorflow workflow

In [ ]:
%env WANDB_PROJECT=distillgpt2Tensorflow

env: WANDB_PROJECT=distillgpt2Tensorflow


In [ ]:
  dataset_comb_tr = pd.read_csv('/content/drive/MyDrive/working directory/tfm/Distilled GPT2 finetuning/datasets/Combined(PAWSX + Tapaco)/dataset_comb_tr.csv')
  dataset_comb_test = pd.read_csv('/content/drive/MyDrive/working directory/tfm/Distilled GPT2 finetuning/datasets/Combined(PAWSX + Tapaco)/dataset_comb_test.csv')
  dataset_comb_val = pd.read_csv('/content/drive/MyDrive/working directory/tfm/Distilled GPT2 finetuning/datasets/Combined(PAWSX + Tapaco)/dataset_comb_val.csv')

In [ ]:
  dataset_comb_tr[dataset_comb_tr['combined'].map(type) != str]

,combined,lan


In [ ]:
MODEL_CHECKPOINT='distilgpt2'

tokenizer = AutoTokenizer.from_pretrained(MODEL_CHECKPOINT)
model = TFAutoModelForCausalLM.from_pretrained(MODEL_CHECKPOINT)

In [ ]:
BATCH_SIZE = 8
NUM_EPOCHS = 5

tokenizer.pad_token = tokenizer.eos_token

In [ ]:
dataset_comb_tr = Dataset.from_pandas(dataset_comb_tr)
dataset_comb_test = Dataset.from_pandas(dataset_comb_test)
dataset_comb_val = Dataset.from_pandas(dataset_comb_val)

In [ ]:
dataset_comb_test

Dataset({
    features: ['combined', 'lan'],
    num_rows: 20153
})

In [ ]:
def tokenize_function(example):
    return tokenizer(example['combined'], truncation=True, max_length=512) # same than block size

In [ ]:
tokenized_train_dataset =   dataset_comb_tr.map(tokenize_function, batched=True)
tokenized_test_dataset =   dataset_comb_test.map(tokenize_function, batched=True)
tokenized_val_dataset =  dataset_comb_val.map(tokenize_function, batched=True)

  0%|          | 0/162 [00:00<?, ?ba/s]

  0%|          | 0/21 [00:00<?, ?ba/s]

  0%|          | 0/21 [00:00<?, ?ba/s]

In [ ]:
tokenized_train_dataset

Dataset({
    features: ['combined', 'lan', 'input_ids', 'attention_mask'],
    num_rows: 161225
})

In [ ]:
data_collator =  DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False,  return_tensors="tf")

In [ ]:
train_ds = tokenized_train_dataset.to_tf_dataset(
    columns=['input_ids', 'attention_mask','labels'],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=8,
	drop_remainder=True,
)
eval_ds = tokenized_train_dataset.to_tf_dataset(
    columns=['input_ids', 'attention_mask','labels'],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=8,
	drop_remainder=True,
)
test_ds = tokenized_train_dataset.to_tf_dataset(
    columns=['input_ids', 'attention_mask','labels'],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=8,
	drop_remainder=True,
)

In [ ]:
trainlist=list(train_ds.as_numpy_iterator())

In [ ]:
input_ids,attencion_mask = trainlist[0]

In [ ]:
model(*trainlist[0][0])

In [ ]:
import os
import psutil
import tensorflow as tf
from itertools import chain
from datasets import load_dataset
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers.schedules import PolynomialDecay
from transformers import AutoTokenizer, TFAutoModelForCausalLM, DataCollatorWithPadding

num_train_steps = len(tf_ds) * NUM_EPOCHS
lr_scheduler = PolynomialDecay(
    initial_learning_rate=5e-5,
    end_learning_rate=0.0,
    decay_steps=num_train_steps)

In [ ]:
opt = Adam(learning_rate=lr_scheduler)

In [ ]:
model.compile(  # Adam with polinomial decay of lr
	optimizer=opt,
	loss=SparseCategoricalCrossentropy(from_logits=True, reduction=tf.keras.losses.Reduction.NONE),
)

#tf.keras.mixed_precision.set_global_policy("mixed_float16")

In [ ]:
model.fit(train_ds, validation_data=eval_ds, epochs=NUM_EPOCHS, steps_per_epoch=len(tf_ds) // BATCH_SIZE)

In [ ]:
optimizer, schedule = create_optimizer(
    init_lr=5e-5,
    num_warmup_steps=1_000,
    num_train_steps=num_train_steps,
    weight_decay_rate=0.01,
)
model.compile(optimizer=optimizer)


In [ ]:
model.fit(train_ds, validation_data=eval_ds)